## Setup

For this notebook to function you will need to generate a BoonAI API key.
Visit https://boonai.app and see the API Key section.Save the key into this
directory and name the file "apikey.json".

You can find additional information reguardig this walkthough in the BoonAI documenation.
https://docs.boonai.app/boonsdk/solutions/product-catalog-similarity


In [ ]:
import boonsdk

app = boonsdk.app_from_keyfile("apikey.json")

## Fetch CSV Data

To get the CSV file open to link below, click on "images.csv" in in the data explorer, and then 
the download button on the far right hand side of screen. This will download the CSV file to your local drive.

https://www.kaggle.com/paramaggarwal/fashion-product-images-dataset

In [ ]:
# Change this to point to your images.csv file.
path_to_csv = "~/Downloads/images.csv"

In [ ]:
# max_assets limits the number of assets to import from the CSV.  This
# allows us to take a small sample from the CSV which has 44k products.
max_assets = 1000

# Create a CsvFileImport instance. This doesn't actually perform the CSV, it simply allows you
# away to configure how the CSV will be parsed.
csvimport = boonsdk.CsvFileImport(path_to_csv, 
                                  uri_column=1,
                                  max_assets=max_assets, 
                                  header=True)
                               

## Inspect CSV Items

Before submitting your CsvFileImport to be imported it's a good idea to check to ensure the CSV 
is being parsed correctly and you picked the proper URI column.  The URI column can contain various 
types of URIs including: HTTP, HTTPS, S3, GS, and AZURE.

In [ ]:
# Look at the first item in the first batch.
item1 = list(csvfile)[0][0]
print(item1.uri)

## Perform The Import

Importing the CSV will create an import job.  You can monitor this job at https://boonai.app or 
use the API to wait on the job to complete.

We're also adding boonai-label-detection to this import to provide us with some predictions to work with later.

In [ ]:
app.asset.import_csv(csvfile, modules=["boonai-label-detection"])

## Building an Image Similarity Query

Once the import has completed we can now do a search. Our goal is to search is to find people wearing a striped shirt and jeans.

First we have to make a similarity query by passing our query image to the <i>app.assets.get_sim_query()</i> function.  
Additionally we're adding a filter on  boonai-label-detection< for the label 'jean' which will filter the results to 
people wearing jeans, as detected by the boonai-label-detection module. 


In [ ]:
img_path = "striped-shirt.jpg"
search = {
    "query": {
        "bool": {
            "must": [
                app.assets.get_sim_query(img_path, min_score=0.75)
            ],
            "filter": [
              boonsdk.search.LabelConfidenceQuery(
                    "boonai-label-detection", ["jean"], 0.1, 1.0
                )   
            ]
        }
    },
    "size": 20
}

## Displaying Results

To display the results we're going to download the web-proxy (smaller lower quality jpeg) and display
using ipyplot.  Additionally we're adding the search score as label on our table of images. 

In [ ]:
import ipyplot
from PIL import Image


images = []
labels = []
for asset in app.assets.search(search):
    thumbnail = asset.get_files(category="web-proxy")[0]
    images.append(Image.open(app.assets.download_file(thumbnail)))
    labels.append(asset.score)

ipyplot.plot_images(images, labels, max_images=20, img_width=100)